In [22]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import random
import os

# --- Configuration ---
import os

BASE_PATH = "C:\\Users\\vumma\\OneDrive\\Desktop\\imp\\GEO Pandas"

# Corrected path: The .shp file is directly in BASE_PATH
SUFFOLK_TRACTS_SHP = os.path.join(BASE_PATH, "tl_2024_36_tract.shp")
POPULATION_DATA_CSV = os.path.join(BASE_PATH, "C:\\Users\\vumma\\OneDrive\\Desktop\\imp\\GEO Pandas\\ACSST5Y2023.S0101-Data.csv")
POVERTY_DATA_CSV = os.path.join(BASE_PATH, "C:\\Users\\vumma\\OneDrive\\Desktop\\imp\\GEO Pandas\\ACSST5Y2023.S1701-Data.csv")

# Define the dot value: How many people does one dot represent?
DOT_VALUE = 100

# FIPS codes (standard for New York State and Suffolk County)
NY_STATE_FIPS = '36'
SUFFOLK_COUNTY_FIPS = '103'

# Initialize gdf_suffolk_tracts to None or an empty GeoDataFrame
# This helps prevent NameError if something fails before it's properly assigned
gdf_suffolk_tracts = None 

# --- Data Loading and Preprocessing ---

print("Step 1: Loading geospatial data (Census Tracts)...")
try:
    print(f"Attempting to load shapefile from: {SUFFOLK_TRACTS_SHP}")
    # Load the shapefile for all NY tracts
    gdf_tracts = gpd.read_file(SUFFOLK_TRACTS_SHP)
    print(f"Shapefile loaded successfully. Total tracts in NY: {len(gdf_tracts)}")
    print("Shapefile columns:", gdf_tracts.columns.tolist())
    print("Sample STATEFP:", gdf_tracts['STATEFP'].head())
    print("Sample COUNTYFP:", gdf_tracts['COUNTYFP'].head())


    # Filter for Suffolk County only using STATEFP and COUNTYFP
    gdf_suffolk_tracts = gdf_tracts[
        (gdf_tracts['STATEFP'] == NY_STATE_FIPS) &
        (gdf_tracts['COUNTYFP'] == SUFFOLK_COUNTY_FIPS)
    ].copy()

    if gdf_suffolk_tracts.empty:
        print(f"WARNING: No tracts found for Suffolk County (FIPS: {SUFFOLK_COUNTY_FIPS}) in the loaded shapefile.")
        print("This could mean the FIPS codes are incorrect, or the shapefile does not contain Suffolk County data.")
        print("Please verify the 'STATEFP' and 'COUNTYFP' values in your shapefile.")
        exit() # Exit if no Suffolk tracts are found, as further steps will fail

    print(f"Filtered to {len(gdf_suffolk_tracts)} census tracts for Suffolk County.")
    print("Columns in Suffolk GeoDataFrame:", gdf_suffolk_tracts.columns.tolist())
    print("Example GEOID in Suffolk GeoDataFrame:", gdf_suffolk_tracts['GEOID'].iloc[0] if not gdf_suffolk_tracts.empty else "N/A")

except FileNotFoundError:
    print(f"ERROR: Shapefile not found at {SUFFOLK_TRACTS_SHP}.")
    print("Please ensure the 'tl_2023_36_tract' folder is in your BASE_PATH and contains 'tl_2023_36_tract.shp'.")
    exit()
except Exception as e:
    print(f"ERROR during shapefile loading or filtering: {e}")
    print("This might be related to GDAL_DATA not being set correctly or a corrupt shapefile.")
    exit() # Exit if shapefile loading fails as it's critical

# Ensure gdf_suffolk_tracts is defined before proceeding
if gdf_suffolk_tracts is None:
    print("Fatal Error: gdf_suffolk_tracts was not defined. Exiting script.")
    exit()


print("\nStep 2: Loading and cleaning attribute data (Population & Poverty)...")
# ... (rest of your code from previous response) ...
try:
    # Load Population Data (S0101)
    pop_df = pd.read_csv(POPULATION_DATA_CSV, encoding='utf-8', skiprows=[1])
    print(f"Loaded Population CSV: {len(pop_df)} rows.")
    print("Population CSV columns:", pop_df.columns.tolist())
    print("Sample Population CSV GEO_ID:", pop_df['GEO_ID'].head())


    pop_df['GEOID'] = pop_df['GEO_ID'].str.replace('1400000US', '')
    pop_df = pop_df[['GEOID', 'S0101_C01_001E']].copy()
    pop_df.rename(columns={'S0101_C01_001E': 'Total_Population'}, inplace=True)
    pop_df['Total_Population'] = pd.to_numeric(pop_df['Total_Population'], errors='coerce').fillna(0)


    # Load Poverty Data (S1701)
    poverty_df = pd.read_csv(POVERTY_DATA_CSV, encoding='utf-8', skiprows=[1])
    print(f"Loaded Poverty CSV: {len(poverty_df)} rows.")
    print("Poverty CSV columns:", poverty_df.columns.tolist())
    print("Sample Poverty CSV GEO_ID:", poverty_df['GEO_ID'].head())

    poverty_df['GEOID'] = poverty_df['GEO_ID'].str.replace('1400000US', '')
    poverty_df = poverty_df[['GEOID', 'S1701_C01_001E', 'S1701_C03_001E']].copy()
    poverty_df.rename(columns={'S1701_C01_001E': 'Poverty_Pop_Total_Determined',
                               'S1701_C03_001E': 'Poverty_Count'}, inplace=True)
    poverty_df['Poverty_Pop_Total_Determined'] = pd.to_numeric(poverty_df['Poverty_Pop_Total_Determined'], errors='coerce').fillna(0)
    poverty_df['Poverty_Count'] = pd.to_numeric(poverty_df['Poverty_Count'], errors='coerce').fillna(0)

    merged_attr_df = pd.merge(pop_df, poverty_df, on='GEOID', how='inner')
    print("Merged attribute data head:")
    print(merged_attr_df.head())
    print(f"Merged attribute data has {len(merged_attr_df)} unique GEOIDs.")

except FileNotFoundError:
    print("ERROR: One or both of your ACS CSV files not found. Check their names and ensure they are in your BASE_PATH.")
    exit()
except Exception as e:
    print(f"ERROR loading or cleaning attribute data: {e}")
    print("Check CSV file paths, encoding, skiprows, and column names (especially 'GEO_ID' and data fields).")
    exit()

print("\nStep 3: Merging attribute data with geospatial data...")
# Now, gdf_suffolk_tracts should definitely be defined from Step 1's successful execution.
# Check if there's any common GEOID before merging
common_geoids = set(gdf_suffolk_tracts['GEOID']).intersection(set(merged_attr_df['GEOID']))
print(f"Number of common GEOIDs between shapefile and attribute data: {len(common_geoids)}")
if not common_geoids:
    print("WARNING: No common GEOIDs found between the shapefile and your attribute data. Merge will result in an empty GeoDataFrame.")
    print("Check if the GEOID format or values are consistent in both datasets.")
    # Consider what to do here: maybe exit, or continue knowing the result will be empty.
    # For now, let's proceed to see the empty result.

gdf_suffolk_tracts = gdf_suffolk_tracts.merge(merged_attr_df, on='GEOID', how='inner')
print(f"Merged GeoDataFrame has {len(gdf_suffolk_tracts)} tracts after attribute merge.")
print("Columns after attribute merge:", gdf_suffolk_tracts.columns.tolist())
# Ensure Total_Population is numeric and handle potential division by zero
gdf_suffolk_tracts['Total_Population'] = pd.to_numeric(gdf_suffolk_tracts['Total_Population'], errors='coerce').fillna(0)
gdf_suffolk_tracts['Poverty_Count'] = pd.to_numeric(gdf_suffolk_tracts['Poverty_Count'], errors='coerce').fillna(0)

# Calculate Poverty Rate
# Fill NaN values (e.g., from division by zero) with 0
gdf_suffolk_tracts['Poverty_Rate'] = (
    (gdf_suffolk_tracts['Poverty_Count'] / gdf_suffolk_tracts['Total_Population']) * 100
).fillna(0)

# Optional: Print a sample to verify the new column
print("\nSample of merged data with new Poverty_Rate column:")
print(gdf_suffolk_tracts[['GEOID', 'Total_Population', 'Poverty_Count', 'Poverty_Rate']].head())
# --- END ADDED LINES ---


# --- Step 4: Generating dot density map points ---
print("\nStep 4: Generating dot density map points. This may take a while for many tracts/dots...")
all_dots = []

# Using tqdm for a progress bar if available (optional)
try:
    from tqdm.auto import tqdm
    iterator = tqdm(gdf_suffolk_tracts.iterrows(), total=len(gdf_suffolk_tracts), desc="Generating dots")
except ImportError:
    iterator = gdf_suffolk_tracts.iterrows()

for index, row in iterator:
    try:
        # Ensure that Total_Population and Poverty_Count are numeric
        total_pop = int(row['Total_Population'])
        poverty_count = int(row['Poverty_Count'])
        tract_geometry = row['geometry']

        if not tract_geometry.is_valid:
            # print(f"Skipping invalid geometry for GEOID: {row['GEOID']} (geometry is invalid).")
            continue
        if tract_geometry.is_empty:
            # print(f"Skipping empty geometry for GEOID: {row['GEOID']} (geometry is empty).")
            continue
        
        # Ensure that geometry is a Polygon (or MultiPolygon), not a Point or Line
        if tract_geometry.geom_type not in ['Polygon', 'MultiPolygon']:
            # print(f"Skipping non-polygon geometry type '{tract_geometry.geom_type}' for GEOID: {row['GEOID']}.")
            continue

        # Ensure that poverty_count does not exceed total_pop to avoid negative general dots
        poverty_count_adj = min(poverty_count, total_pop)
        num_poverty_dots = poverty_count_adj // DOT_VALUE
        num_general_dots = (total_pop - poverty_count_adj) // DOT_VALUE

        # Generate random points within the polygon for general population
        for _ in range(num_general_dots):
            minx, miny, maxx, maxy = tract_geometry.bounds
            point = None
            for i in range(100): # Increased tries for robustness
                random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
                if tract_geometry.contains(random_point):
                    point = random_point
                    break
            if point:
                all_dots.append({'geometry': point, 'type': 'General Population', 'GEOID': row['GEOID']})
            # else:
            #     print(f"Warning: Could not place general dot in {row['GEOID']} after 100 tries.")


        # Generate random points within the polygon for poverty population (red dots)
        for _ in range(num_poverty_dots):
            minx, miny, maxx, maxy = tract_geometry.bounds
            point = None
            for i in range(100): # Increased tries for robustness
                random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
                if tract_geometry.contains(random_point):
                    point = random_point
                    break
            if point:
                all_dots.append({'geometry': point, 'type': 'Poverty', 'GEOID': row['GEOID']})
            # else:
            #     print(f"Warning: Could not place poverty dot in {row['GEOID']} after 100 tries.")


    except Exception as e:
        print(f"Error processing tract {row['GEOID']}: {e}. Data: Pop={total_pop}, Poverty={poverty_count}")
        continue

if not all_dots:
    print("\nNo dots were generated. This can happen if 'DOT_VALUE' is too high relative to population/poverty counts in tracts, or if there are issues with geometries or data values.")
    print("Please check: 1. 'DOT_VALUE' setting. 2. Values in 'Total_Population' and 'Poverty_Count' columns. 3. Geometry validity/type of your tracts.")
    exit()

# Create a new GeoDataFrame from the generated dots
gdf_dots = gpd.GeoDataFrame(all_dots, crs=gdf_suffolk_tracts.crs)
print(f"\nGenerated {len(gdf_dots)} dots.")
print("Dot GeoDataFrame head:")
print(gdf_dots.head())
print("Dot types counts:\n", gdf_dots['type'].value_counts())

# --- Step 5: Export to Shapefile ---
OUTPUT_SHAPEFILE_PATH = os.path.join(BASE_PATH, "Suffolk_Population_Poverty_Dots.shp")

print(f"\nStep 5: Exporting dots to shapefile: {OUTPUT_SHAPEFILE_PATH}")
try:
    output_dir = os.path.dirname(OUTPUT_SHAPEFILE_PATH)
    os.makedirs(output_dir, exist_ok=True)

    gdf_dots.to_file(OUTPUT_SHAPEFILE_PATH, driver='ESRI Shapefile')
    print("Shapefile exported successfully! You can now load 'Suffolk_Population_Poverty_Dots.shp' into QGIS.")
except Exception as e:
    print(f"Error exporting shapefile: {e}")

print("\nPython script finished.")

Step 1: Loading geospatial data (Census Tracts)...
Attempting to load shapefile from: C:\Users\vumma\OneDrive\Desktop\imp\GEO Pandas\tl_2024_36_tract.shp
Shapefile loaded successfully. Total tracts in NY: 5411
Shapefile columns: ['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'GEOIDFQ', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry']
Sample STATEFP: 0    36
1    36
2    36
3    36
4    36
Name: STATEFP, dtype: object
Sample COUNTYFP: 0    093
1    093
2    093
3    081
4    081
Name: COUNTYFP, dtype: object
Filtered to 385 census tracts for Suffolk County.
Columns in Suffolk GeoDataFrame: ['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'GEOIDFQ', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry']
Example GEOID in Suffolk GeoDataFrame: 36103158320

Step 2: Loading and cleaning attribute data (Population & Poverty)...
Loaded Population CSV: 372 rows.
Population CSV columns: ['GEO_ID', 'NAME', 'S0101_C01_

Generating dots:   0%|          | 0/371 [00:00<?, ?it/s]


Generated 14250 dots.
Dot GeoDataFrame head:
                 geometry                type        GEOID
0   POINT (-73.01 40.912)  General Population  36103158320
1  POINT (-73.006 40.894)  General Population  36103158320
2  POINT (-73.012 40.907)  General Population  36103158320
3  POINT (-73.008 40.895)  General Population  36103158320
4  POINT (-73.017 40.915)  General Population  36103158320
Dot types counts:
 type
General Population    14250
Name: count, dtype: int64

Step 5: Exporting dots to shapefile: C:\Users\vumma\OneDrive\Desktop\imp\GEO Pandas\Suffolk_Population_Poverty_Dots.shp
Error exporting shapefile: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\vumma\\OneDrive\\Desktop\\imp\\GEO Pandas\\Suffolk_Population_Poverty_Dots.shp'

Python script finished.


In [26]:
import geopandas as gpd
import pandas as pd
import numpy as np
import random # For random point generation

BASE_PATH = "C:\\Users\\vumma\\OneDrive\\Desktop\\imp\\GEO Pandas"

# Corrected path: The .shp file is directly in BASE_PATH
SUFFOLK_TRACTS_SHP = os.path.join(BASE_PATH, "tl_2024_36_tract.shp")
POPULATION_DATA_CSV = os.path.join(BASE_PATH, "C:\\Users\\vumma\\OneDrive\\Desktop\\imp\\GEO Pandas\\ACSST5Y2023.S0101-Data.csv")
POVERTY_DATA_CSV = os.path.join(BASE_PATH, "C:\\Users\\vumma\\OneDrive\\Desktop\\imp\\GEO Pandas\\ACSST5Y2023.S1701-Data.csv")

# Ensure gdf_suffolk_tracts has 'Total_Population' and 'Poverty_Count'
# and that they are numeric. Handle potential NaN/None values.
gdf_suffolk_tracts['Total_Population'] = pd.to_numeric(gdf_suffolk_tracts['Total_Population'], errors='coerce').fillna(0)
gdf_suffolk_tracts['Poverty_Count'] = pd.to_numeric(gdf_suffolk_tracts['Poverty_Count'], errors='coerce').fillna(0)

# ... (your existing code before Step 4) ...

print("\nStep 4: Generating Dot Density Map...")

# Define your dot value (e.g., 1 dot represents X people)
POPULATION_DOT_VALUE = 100  # 1 dot = 100 people
POVERTY_DOT_VALUE = 1     # 1 dot = 50 people in poverty (can be adjusted)

# Initialize an empty list to store all the generated dots
all_dots = []

# --- DEBUGGING PRINTS START ---
print(f"Debugging dot generation with POPULATION_DOT_VALUE={POPULATION_DOT_VALUE} and POVERTY_DOT_VALUE={POVERTY_DOT_VALUE}")
# --- DEBUGGING PRINTS END ---

# Iterate over each row in the gdf_suffolk_tracts (each census tract)
for index, row in gdf_suffolk_tracts.iterrows():
    # Get the geometry of the current tract
    tract_geometry = row.geometry
    tract_geoid = row.GEOID # Assuming GEOID is the unique identifier

    # Ensure population and poverty counts are numeric and handle potential NaN/None values.
    # These lines are crucial and should be run *before* the loop, or ensure they are done correctly.
    # I see you have them before the loop in the screenshot, which is good.
    total_pop = row['Total_Population']
    poverty_count = row['Poverty_Count']

    # --- DEBUGGING PRINTS START ---
    # Print for a few tracts to avoid too much output, maybe first 5, then every 50th
    if index < 5 or index % 50 == 0:
        print(f"\n--- Tract GEOID: {tract_geoid} (Index: {index}) ---")
        print(f"  Total Population: {total_pop}")
        print(f"  Poverty Count: {poverty_count}")
    # --- DEBUGGING PRINTS END ---

    # --- Generate General Population Dots ---
    num_population_dots = int(total_pop / POPULATION_DOT_VALUE)

    # --- DEBUGGING PRINTS START ---
    if index < 5 or index % 50 == 0:
        print(f"  Calculated Population Dots: {num_population_dots}")
    # --- DEBUGGING PRINTS END ---

    points_in_tract_pop = []
    attempts = 0
    while len(points_in_tract_pop) < num_population_dots and attempts < num_population_dots * 10:
        x = random.uniform(tract_geometry.bounds[0], tract_geometry.bounds[2])
        y = random.uniform(tract_geometry.bounds[1], tract_geometry.bounds[3])
        point = gpd.points_from_xy([x], [y])[0]
        if tract_geometry.contains(point):
            points_in_tract_pop.append({'geometry': point, 'GEOID': tract_geoid, 'type': 'General Population'})
        attempts += 1
    
    all_dots.extend(points_in_tract_pop)

    # --- Generate Poverty Dots ---
    num_poverty_dots = int(poverty_count / POVERTY_DOT_VALUE)

    # --- DEBUGGING PRINTS START ---
    if index < 5 or index % 50 == 0:
        print(f"  Calculated Poverty Dots: {num_poverty_dots}")
    # --- DEBUGGING PRINTS END ---

    points_in_tract_pov = []
    attempts = 0
    while len(points_in_tract_pov) < num_poverty_dots and attempts < num_poverty_dots * 10:
        x = random.uniform(tract_geometry.bounds[0], tract_geometry.bounds[2])
        y = random.uniform(tract_geometry.bounds[1], tract_geometry.bounds[3])
        point = gpd.points_from_xy([x], [y])[0]
        if tract_geometry.contains(point):
            points_in_tract_pov.append({'geometry': point, 'GEOID': tract_geoid, 'type': 'Poverty'})
        attempts += 1
    
    all_dots.extend(points_in_tract_pov)


# Create a GeoDataFrame from all generated dots
if all_dots:
    gdf_dots = gpd.GeoDataFrame(all_dots, crs=gdf_suffolk_tracts.crs)
    gdf_dots = gdf_dots[['geometry', 'GEOID', 'type']]

    # --- DEBUGGING PRINTS START ---
    print("\n--- Final gdf_dots 'type' column counts ---")
    print(gdf_dots['type'].value_counts())
    print("------------------------------------------")
    # --- DEBUGGING PRINTS END ---

    DOTS_OUTPUT_SHP = os.path.join(BASE_PATH, "Suffolk_Population_Poverty_Dots.shp")

    print(f"Step 5: Exporting dots to shapefile: {DOTS_OUTPUT_SHP}")
    gdf_dots.to_file(DOTS_OUTPUT_SHP)
    print("Shapefile exported successfully! You can now load it into QGIS.")
else:
    print("No dots were generated. Check your population/poverty data and DOT_VALUEs.")


Step 4: Generating Dot Density Map...
Debugging dot generation with POPULATION_DOT_VALUE=100 and POVERTY_DOT_VALUE=1

--- Tract GEOID: 36103158320 (Index: 0) ---
  Total Population: 5874
  Poverty Count: 11.2
  Calculated Population Dots: 58
  Calculated Poverty Dots: 11

--- Tract GEOID: 36103146608 (Index: 1) ---
  Total Population: 2839
  Poverty Count: 11.4
  Calculated Population Dots: 28
  Calculated Poverty Dots: 11

--- Tract GEOID: 36103158403 (Index: 2) ---
  Total Population: 2582
  Poverty Count: 11.4
  Calculated Population Dots: 25
  Calculated Poverty Dots: 11

--- Tract GEOID: 36103158407 (Index: 3) ---
  Total Population: 5665
  Poverty Count: 1.3
  Calculated Population Dots: 56
  Calculated Poverty Dots: 1

--- Tract GEOID: 36103158408 (Index: 4) ---
  Total Population: 3753
  Poverty Count: 5.2
  Calculated Population Dots: 37
  Calculated Poverty Dots: 5

--- Tract GEOID: 36103190604 (Index: 50) ---
  Total Population: 5494
  Poverty Count: 7.7
  Calculated Popula